# Evaluate MobileNet V2

Install Weights and Biases since it is not in Google Colab by default

In [1]:
!pip install wandb -qqq

Mount Google Drive to access images

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from wandb.keras import WandbCallback
import wandb

In [4]:
# Initialise Weights and Biases API
# Hyperparameters

# previous config:
# config={
#     'batch_size': 128,  # tried 64, 256
#     'epochs': 10,  # tried 20
#     'seed': 0,  # tried 42
#     'learning_rate': 0.0001,  # tried 0.001, 0.00005
#     'dropout_rate': 0.2,  # tried 0.5 and no dropout
#     'bn_momentum': 0.9,  # tried default of 0.99
#     'fc1_num_neurons': 1024,
#     'fc2_num_neurons': 512,
#     'fc3_num_neurons': 256,
#     'hidden_activation': 'relu',
#     'output_activation': 'sigmoid',  # tried softmax
#     'loss_function': 'binary_crossentropy',
#     'metrics': ['accuracy'],
# }

# tried sgd
# tried splitting data into train-val-test instead of using test set as val set

defaults = {
    'epochs': 20,
    'batch_size': 64,
    'fc1_num_neurons': 1024,
    'fc2_num_neurons': 512,
    'fc3_num_neurons': 256,
    'seed': 7,
    'learning_rate': 1e-3,
    'optimizer': 'adam',
    'hidden_activation': 'relu',
    'output_activation': 'sigmoid',
    'loss_function': 'binary_crossentropy',
    'metrics': ['accuracy'],
}

wandb.login()

run = wandb.init(
    name='mobilenet_v2',
    project='cz4042',
    config=defaults,
)

config = wandb.config

wandb: Currently logged in as: burntice (use `wandb login --relogin` to force relogin)


Import relevant libraries

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array

import pandas as pd
import tensorflow as tf

In [6]:
tf.random.set_seed(config.seed)

Load data

In [7]:
# Load dataset as dataframe
base_path = '/content/drive/My Drive/NN Project/'
df = pd.read_csv(base_path + 'aligned_gender.txt', sep='\t')
df['datadir'] = base_path + df['datadir'].astype(str)

# Train test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=config.seed)

In [8]:
print(train_df.shape)
print(test_df.shape)

(9755, 2)
(2439, 2)


In [9]:
train_df['gender'].value_counts() / train_df.shape[0] * 100

0    52.598667
1    47.401333
Name: gender, dtype: float64

In [10]:
# Load images into keras image generator 
datagen_train = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)
datagen_test = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
)

train_generator = datagen_train.flow_from_dataframe(
    dataframe=train_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

test_generator = datagen_test.flow_from_dataframe(
    dataframe=test_df,
    x_col='datadir',
    y_col='gender',
    batch_size=config.batch_size,
    seed=config.seed,
    shuffle=True,
    class_mode='raw',
    target_size=(224,224),
)

Found 9755 validated image filenames.
Found 2439 validated image filenames.


In [11]:
mobile_net_v2 = tf.keras.applications.MobileNetV2(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    input_shape=(224,224,3),
)
mobile_net_v2.trainable = False

fc1 = tf.keras.layers.Dense(
    config.fc1_num_neurons,
    activation=config.hidden_activation,
)

fc2 = tf.keras.layers.Dense(
    config.fc2_num_neurons,
    activation=config.hidden_activation,
)

fc3 = tf.keras.layers.Dense(
    config.fc3_num_neurons,
    activation=config.hidden_activation,
)

model = tf.keras.models.Sequential([
    mobile_net_v2,
    tf.keras.layers.Flatten(),
    tf.keras.layers.BatchNormalization(),
    fc1,
    tf.keras.layers.BatchNormalization(),
    fc2,
    tf.keras.layers.BatchNormalization(),
    fc3,
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation=config.output_activation),
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 1280)              2257984   
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0         
_________________________________________________________________
batch_normalization (BatchNo (None, 1280)              5120      
_________________________________________________________________
dense (Dense)                (None, 1024)              1311744   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2

In [12]:
# Compile model 
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config.learning_rate),
    loss=config.loss_function,
    metrics=config.metrics,
)

In [13]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=config.epochs,
    callbacks=[WandbCallback()],
)

Epoch 1/20
153/153 [==============================] - 146s 956ms/step - loss: 0.4091 - accuracy: 0.8226 - val_loss: 0.3268 - val_accuracy: 0.8573
Epoch 2/20
153/153 [==============================] - 147s 958ms/step - loss: 0.2033 - accuracy: 0.9195 - val_loss: 0.3131 - val_accuracy: 0.8815
Epoch 3/20
153/153 [==============================] - 146s 954ms/step - loss: 0.1355 - accuracy: 0.9482 - val_loss: 0.3392 - val_accuracy: 0.8733
Epoch 4/20
153/153 [==============================] - 146s 957ms/step - loss: 0.0958 - accuracy: 0.9644 - val_loss: 0.3428 - val_accuracy: 0.8795
Epoch 5/20
153/153 [==============================] - 145s 951ms/step - loss: 0.0770 - accuracy: 0.9717 - val_loss: 0.3614 - val_accuracy: 0.8864
Epoch 6/20
153/153 [==============================] - 145s 949ms/step - loss: 0.0736 - accuracy: 0.9724 - val_loss: 0.3777 - val_accuracy: 0.8868
Epoch 7/20
153/153 [==============================] - 146s 951ms/step - loss: 0.0524 - accuracy: 0.9823 - val_loss: 0.4463 -

In [14]:
results = model.evaluate(
    test_generator,
    callbacks=[WandbCallback()],
)

39/39 [==============================] - 28s 707ms/step - loss: 0.5182 - accuracy: 0.8827


In [15]:
save_path = base_path + '5. Saved Weights and Models/Evaluation/mobilenetv2_eval.h5'
model.save(save_path)

Let Weights and Biases know that this run is complete.

In [16]:
run.finish()

epoch,19
loss,0.01587
accuracy,0.99446
val_loss,0.51823
val_accuracy,0.88274
_step,19
_runtime,2910
_timestamp,1606105365
best_val_loss,0.31311
best_epoch,1


epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▄▃▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▅▆▇▇▇██▇███████████
val_loss,▁▁▂▂▃▃▅▆▅▅▅▆▆▇▇▆█▇██
val_accuracy,▁▆▄▅▇▇▇▆▆▅▆▇▇▆▇█▆▆█▆
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
